In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
training_data=pd.read_csv('../input/train.csv',nrows=2000000)
test_data=pd.read_csv('../input/test.csv')

In [ ]:
training_data

Planning to use Neural network for regression prediction. Training data is huge with 55M rows so neural network should be a better option, XGBoost might get stuck in computations. Now, difficult to calculate exact distance from longitude and latitude (in km or miles), so better to simply use difference in longitude and latitude, pickup points, number of passengers. The data is all relative so absolute distance won't be of much use.

In [ ]:
X_train=training_data.copy()
X_test=test_data.copy()
Y_train=training_data.copy()

In [ ]:
X_train['pickup_datetime']=pd.to_datetime(X_train['pickup_datetime'])
X_train['hour']=X_train['pickup_datetime'].dt.hour
# X_train['dayofweek']=X_train['pickup_datetime'].dt.dayofweek
X_train['latitude_distance']=abs(X_train['dropoff_latitude']- X_train['pickup_latitude'])
X_train['longitude_distance']=abs(X_train['dropoff_longitude']- X_train['pickup_longitude'])
X_train=X_train.drop(columns=['dropoff_longitude','dropoff_latitude','key','fare_amount','pickup_datetime','passenger_count'])

In [ ]:
X_test['pickup_datetime']=pd.to_datetime(X_test['pickup_datetime'])
X_test['hour']=X_test['pickup_datetime'].dt.hour
# X_test['dayofweek']=X_test['pickup_datetime'].dt.dayofweek
X_test['latitude_distance']=abs(X_test['dropoff_latitude']- X_test['pickup_latitude'])
X_test['longitude_distance']=abs(X_test['dropoff_longitude']- X_test['pickup_longitude'])
X_test=X_test.drop(columns=['dropoff_longitude','dropoff_latitude','key','pickup_datetime','passenger_count'])

In [ ]:
Y_train=Y_train.drop(columns=['key','pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count'])

In [ ]:
"""
from keras import models
from keras import layers
from keras import optimizers
from keras.layers import Dropout

model=models.Sequential()
model.add(layers.Dense(512,activation='relu',input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(layers.Dense(512,activation='relu'))
model.add(Dropout(0.2))
model.add(layers.Dense(1))

rmsprop=optimizers.RMSprop(lr=0.001)

model.compile(optimizer=rmsprop,loss='mse',metrics=['mae'])

model.fit(X_train,Y_train,epochs=4,batch_size=512)

Y_pred=model.predict(X_test)
"""

In [ ]:
import xgboost as xgb

model=xgb.XGBRegressor()
model.fit(X_train, Y_train)

Y_pred=model.predict(data=X_test)

In [ ]:
from matplotlib import pyplot as plt
from xgboost import plot_importance

plot_importance(model)
plt.show()

Interesting insight- so passenger count, day of week are not having much importance in deciding fare, most important are distance and pickup point. Also, to some extent- hour has some importance in fares.

In [ ]:
submission=test_data.copy()
submission=submission.drop(columns=['pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count'])
submission['fare_amount']=Y_pred

In [ ]:
submission.to_csv('sample_submission.csv',index=False)